# Notebook Pelatihan Utama - Pengenalan Ucapan Disartria (KAGGLE VERSION)
**Version:** 20260116_0809\n
**Tujuan:** Analisis Perbandingan **Lightweight CNN-STFT** (Diusulkan) vs **Model Transfer Learning**.
**Platform:** Kaggle Kernels (GPU T4 x2).
**Strategy:** Subject-Independent Split (Verified).

## 🆕 Log Perubahan (Changelog)

*   **Fix**: Resolved `axis 1 out of bounds` error for NASNetMobile.
*   **Fix**: **Model Size Analysis** now correctly reports Inference Size (weights only), excluding Optimizer state (122MB -> ~400KB).
*   **Feature**: Added **Thesis-Ready Visualization** suite (CM, ROC, PRC, Learning Curves saved as PNG).
*   **Feature**: Added **Comparison Plots** (Accuracy & Time Bar Charts) at the end of the notebook.
*   **Feature**: Added **Extended CSV Logs** (Predictions with file names for Error Analysis).
*   **Optimization**: Fully implemented **Paper 2 Alignment** (16kHz, Librosa STFT).
*   **Correction**: Data Prep uses **Strict Paired Logic** & **STFT Spectrograms**.
*   **Feature**: Added **Full Validation Logging** (`_val_report.json`, `_val_eval.json`, `_val_cm.png`).
*   **Correction**: Removed individual ROC/PRC plots (Paper 2 Strict - Combined Only).
*   **Feature**: Added Overall Performance Bar & Combined Learning/CM Plots.
\n
## 📋 Panduan Setup Kaggle
1. **Add Data**: Upload folder `backend` anda sebagai Dataset (beri nama `thesis-backend` misalnya).
2. **Add Data**: Cari dataset `UASpeech` dan `TORGO` (atau upload zip-nya jika punya privasi).
3. **Internet**: Aktifkan Internet di menu Settings (kanan) jika perlu download via `gdown`.

In [ ]:
# 1. Setup Environment & Path (Kaggle Symlink Fix)
%load_ext tensorboard
import os
import sys
import glob

print("🚀 Memulai Setup Kaggle Environment...")

# A. Cari file 'config.py' dimanapun dia berada
config_path = None
for root, dirs, files in os.walk('/kaggle/input'):
    if 'config.py' in files:
        config_path = os.path.join(root, 'config.py')
        break

if config_path:
    print(f"✅ Ditemukan Config di: {config_path}")
    source_dir = os.path.dirname(config_path)
    
    # B. Buat Symlink 'src' di Working Directory
    # Tujuannya agar 'from src import config' SELALU jalan, tidak peduli struktur aslinya rusak/flatten
    target_link = '/kaggle/working/src'
    if os.path.exists(target_link):
        if os.path.islink(target_link):
            os.unlink(target_link)
        else:
            import shutil
            shutil.rmtree(target_link)
            
    os.symlink(source_dir, target_link)
    print(f"🔗 Symlink dibuat: {target_link} -> {source_dir}")
    
    # C. Tambah Working Dir ke Sys Path
    if '/kaggle/working' not in sys.path:
        sys.path.append('/kaggle/working')
else:
    print("❌ FATAL: File 'config.py' tidak ditemukan di Input manapun!")
    print("   Pastikan Anda sudah 'Add Data' folder backend.")

# D. Setup Output Paths
OUTPUT_ROOT = '/kaggle/working'
LOCAL_DATA_ROOT = '/kaggle/working/data'
os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)

print("Environment Siap!")

In [ ]:
# 2. Install Dependencies
!pip install -q tensorflow-io
!pip install -q pandas matplotlib seaborn scikit-learn librosa

In [ ]:
# 3. Import Modul Proyek
try:
    from src import config, data_loader, models, trainer
    print("✅ Modul berhasil diimport: config, data_loader, models, trainer")

    # Override Config untuk Kaggle Output
    config.MODELS_DIR = os.path.join(OUTPUT_ROOT, 'models')
    config.OUTPUTS_DIR = os.path.join(OUTPUT_ROOT, 'outputs')
    os.makedirs(config.MODELS_DIR, exist_ok=True)
    os.makedirs(config.OUTPUTS_DIR, exist_ok=True)
    print(f"📂 Output Directory set to: {config.OUTPUTS_DIR}")

except ImportError as e:
    print(f"❌ Gagal import modul: {e}")
    print("Pastikan 'backend' terdeteksi dengan benar.")

In [ ]:
# 3.5 Visualization Helpers (From Paper 2 - CLONED)\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, classification_report\nimport numpy as np\nimport pandas as pd\nimport os\n\ndef smooth_curve(points, factor=0.8):\n    \"\"\"Membuat kurva lebih halus menggunakan exponential moving average.\"\"\"\n    smoothed = []\n    for point in points:\n        if smoothed:\n            # Basic EMA\n            smoothed.append(smoothed[-1] * factor + point * (1 - factor))\n        else:\n            smoothed.append(point)\n    return smoothed\n\ndef plot_learning_curve(history, model_name, run_name):\n    # 1. Grafik Kurva Pembelajaran (ASLI / TANPA SMOOTHING)\n    sns.set_style(\"whitegrid\")\n    fig_learning_asli, axs_learning_asli = plt.subplots(1, 2, figsize=(15, 5))\n    fig_learning_asli.suptitle(f'Kurva Pembelajaran (Asli): {model_name}', fontsize=16)\n\n    axs_learning_asli[0].plot(history['accuracy'], '-', label='Akurasi Training', linewidth=2)\n    axs_learning_asli[0].plot(history['val_accuracy'], '-', label='Akurasi Validasi', linewidth=2)\n    axs_learning_asli[0].set_title('Grafik Akurasi (Asli)')\n    axs_learning_asli[0].set_xlabel('Epoch'); axs_learning_asli[0].set_ylabel('Akurasi')\n    axs_learning_asli[0].legend(loc='lower right')\n\n    axs_learning_asli[1].plot(history['loss'], '-', label='Loss Training', linewidth=2)\n    axs_learning_asli[1].plot(history['val_loss'], '-', label='Loss Validasi', linewidth=2)\n    axs_learning_asli[1].set_title('Grafik Loss (Asli)')\n    axs_learning_asli[1].set_xlabel('Epoch'); axs_learning_asli[1].set_ylabel('Loss')\n    axs_learning_asli[1].legend(loc='upper right')\n    \n    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_learning_asli.png\"))\n    plt.show()\n\n    # 2. Grafik Kurva Pembelajaran (DENGAN SMOOTHING)\n    fig_learning_smooth, axs_learning_smooth = plt.subplots(1, 2, figsize=(15, 5))\n    fig_learning_smooth.suptitle(f'Kurva Pembelajaran (Smoothed): {model_name}', fontsize=16)\n\n    smoothed_accuracy = smooth_curve(history['accuracy'])\n    smoothed_val_accuracy = smooth_curve(history['val_accuracy'])\n    smoothed_loss = smooth_curve(history['loss'])\n    smoothed_val_loss = smooth_curve(history['val_loss'])\n\n    axs_learning_smooth[0].plot(history['accuracy'], '-', label='Akurasi Training (Asli)', alpha=0.3)\n    axs_learning_smooth[0].plot(history['val_accuracy'], '-', label='Akurasi Validasi (Asli)', alpha=0.3)\n    axs_learning_smooth[0].plot(smoothed_accuracy, '-', label='Akurasi Training (Smoothed)', linewidth=2)\n    axs_learning_smooth[0].plot(smoothed_val_accuracy, '-', label='Akurasi Validasi (Smoothed)', linewidth=2)\n    axs_learning_smooth[0].set_title('Grafik Akurasi (Smoothed)')\n    axs_learning_smooth[0].set_xlabel('Epoch'); axs_learning_smooth[0].set_ylabel('Akurasi')\n    axs_learning_smooth[0].legend(loc='lower right')\n\n    axs_learning_smooth[1].plot(history['loss'], '-', label='Loss Training (Asli)', alpha=0.3)\n    axs_learning_smooth[1].plot(history['val_loss'], '-', label='Loss Validasi (Asli)', alpha=0.3)\n    axs_learning_smooth[1].plot(smoothed_loss, '-', label='Loss Training (Smoothed)', linewidth=2)\n    axs_learning_smooth[1].plot(smoothed_val_loss, '-', label='Loss Validasi (Smoothed)', linewidth=2)\n    axs_learning_smooth[1].set_title('Grafik Loss (Smoothed)')\n    axs_learning_smooth[1].set_xlabel('Epoch'); axs_learning_smooth[1].set_ylabel('Loss')\n    axs_learning_smooth[1].legend(loc='upper right')\n\n    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_learning_smooth.png\"))\n    plt.show()\n\ndef plot_confusion_matrix(y_true, y_pred, classes, model_name, run_name, suffix=''):\n    # Paper 2 Style: Heatmap with Count + Percentage\n    cm = confusion_matrix(y_true, y_pred)\n    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]\n    annot_labels = (np.asarray([\"{0:d}\\n({1:.1%})\".format(value, P_value)\n                                  for value, P_value in zip(cm.flatten(), cm_percent.flatten())])\n                    ).reshape(cm.shape)\n    plt.figure(figsize=(8, 6))\n    sns.heatmap(cm_percent, annot=annot_labels, fmt='', cmap='Blues',\n                xticklabels=classes, yticklabels=classes)\n    plt.title(f'Confusion Matrix {suffix}: {model_name}', fontsize=14)\n    plt.ylabel('Label Aktual'); plt.xlabel('Label Prediksi')\n    plt.tight_layout()\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_cm{suffix}.png\"))\n    plt.show()\n\ndef plot_roc_curve(y_true, y_pred_probs, model_name, run_name):\n    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)\n    roc_auc = auc(fpr, tpr)\n    plt.figure(figsize=(8, 6))\n    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')\n    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')\n    plt.legend(loc=\"lower right\")\n    plt.title(f'ROC: {model_name}')\n    plt.grid(True)\n    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_roc.png\"))\n    plt.close()\n\ndef plot_pr_curve(y_true, y_pred_probs, model_name, run_name):\n    precision, recall, _ = precision_recall_curve(y_true, y_pred_probs)\n    plt.figure(figsize=(8, 6))\n    plt.plot(recall, precision, color='purple', lw=2)\n    plt.title(f'PR Curve: {model_name}')\n    plt.xlabel('Recall'); plt.ylabel('Precision')\n    plt.grid(True)\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_prc.png\"))\n    plt.close()\n\ndef plot_class_report(y_true, y_pred, classes, model_name, run_name, suffix=''):\n    # Paper 2 Style: Detailed Bar Chart per Class\n    report_dict = classification_report(y_true, y_pred, target_names=classes, output_dict=True)\n    df = pd.DataFrame(report_dict).transpose().drop(['accuracy', 'macro avg', 'weighted avg'])\n    df = df.reset_index().rename(columns={'index':'class'}).melt(id_vars='class', value_vars=['precision','recall','f1-score'])\n    \n    plt.figure(figsize=(10, 6))\n    ax = sns.barplot(x='class', y='value', hue='variable', data=df, palette='viridis')\n    plt.title(f'Grafik Laporan Klasifikasi per Kelas {suffix}: {model_name}', fontsize=14)\n    plt.ylim(0, 1.1)\n    plt.xlabel('Kelas'); plt.ylabel('Skor')\n    plt.legend(title='Metrik')\n    for p in ax.patches:\n        ax.annotate(f\"{p.get_height():.2f}\", (p.get_x() + p.get_width() / 2., p.get_height()),\n                      ha='center', va='center', xytext=(0, 9), textcoords='offset points')\n                      \n    plt.tight_layout()\n    plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"{run_name}_report_bar{suffix}.png\"))\n    plt.close()\n

In [ ]:
# 4. Persiapan Data (Kaggle Auto-Detect or Gdown)
import shutil
import subprocess
import gdown

# IDs Google Drive (Backup jika file tidak ada di Kaggle Dataset)
UASPEECH_ID = '1L17F0SAkRk3rEjHDUyToLUvNp99sNMvE'
TORGO_ID = '1YU7aCqa4qyn75XRdFPAWEqVv_1Qpl9cG'

def setup_dataset_kaggle(name, file_id, extract_path):
    print(f"\n--- Setup Dataset: {name} ---")
    # ... (Same setup logic) ...
    # 1. Cek di /kaggle/input (Siapa tau user sudah add data)
    candidates = glob.glob(f'/kaggle/input/**/*{name}*', recursive=True)
    potential_dirs = [c for c in candidates if os.path.isdir(c)]
    for p in potential_dirs:
        if os.path.basename(p).lower() == name.lower() or os.path.basename(p).lower() == f"{name}_smalldataset".lower():
             print(f"✅ Ditemukan Dataset di Input: {p}")
             return p
    # 3. Jika tidak ketemu di Input, Coba Download (Gdown)
    print(f"⚠️ {name} tidak ditemukan di ke Kaggle Input. Mencoba download via Gdown...")
    local_zip_path = os.path.join(extract_path, f"{name}.zip")
    target_extract = os.path.join(extract_path, name)
    if os.path.exists(target_extract):
         print(f"✅ Dataset sudah ada di Working Dir: {target_extract}")
         return target_extract
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, local_zip_path, quiet=False)
    print(f"Mengekstrak {name}...")
    subprocess.check_call(['unzip', '-o', '-q', local_zip_path, '-d', extract_path])
    print(f"✅ {name} Selesai diekstrak.")
    if name == 'TORGO' and not os.path.exists(target_extract):
         alt = os.path.join(extract_path, 'TORGO_smalldataset')
         if os.path.exists(alt): return alt
    return target_extract

# Jalankan Setup
uaspeech_path = setup_dataset_kaggle('UASpeech', UASPEECH_ID, LOCAL_DATA_ROOT)
torgo_path = setup_dataset_kaggle('TORGO', TORGO_ID, LOCAL_DATA_ROOT)

# ---------------------------------------------------------
# LOADING DATA
# ---------------------------------------------------------
print("\nMemuat Path File...")
uaspeech_files, uaspeech_labels, uaspeech_speakers = data_loader.get_file_paths(uaspeech_path, 'UASpeech')
torgo_files, torgo_labels, torgo_speakers = data_loader.get_file_paths(torgo_path, 'TORGO')

# --- GENERATE DATASET STATS FOR DASHBOARD ---
import json
print("Generating Dataset Statistics...")
def get_stats(name, files, labels, speakers):
    unique_lbl = list(set(labels)); counts = {l: 0 for l in unique_lbl}
    for l in labels: counts[l] += 1
    summary = []
    for l in unique_lbl:
        cat = "Dysarthric" if l == 1 else "Control"; total = counts[l]
        summary.append({"category": cat, "speakers": len(set(speakers)), "totalRaw": total, "trainRaw": int(total * 0.8), "testRaw": total - int(total * 0.8)})
    return {"name": name, "stats": {"samples": f"{len(files):,}", "classes": str(len(unique_lbl)), "avgLen": "N/A"}, "summaryData": summary}
stats_export = {"uaspeech": get_stats('UASpeech', uaspeech_files, uaspeech_labels, uaspeech_speakers), "torgo": get_stats('TORGO', torgo_files, torgo_labels, torgo_speakers)}
with open(os.path.join(config.OUTPUTS_DIR, "dataset_stats.json"), 'w') as f: json.dump(stats_export, f, indent=4)

# --- GENERATE REAL EDA SAMPLES (Audio + Signals + STFT) ---
print("Generating EDA Samples (Waveform & Spectrogram data)...")
import random; import shutil; import librosa; import numpy as np

samples_out_dir = os.path.join(config.OUTPUTS_DIR, "samples")
os.makedirs(samples_out_dir, exist_ok=True)
eda_export = {}

# Iterate over both datasets
for ds_name, (ds_files, ds_labels, ds_speakers) in [('uaspeech', (uaspeech_files, uaspeech_labels, uaspeech_speakers)), ('torgo', (torgo_files, torgo_labels, torgo_speakers))]:
    eda_export[ds_name] = {'dysarthric': [], 'control': []}
    # Binary Classification Logic
    indices_dys = [i for i, x in enumerate(ds_labels) if x == 1]
    indices_ctrl = [i for i, x in enumerate(ds_labels) if x != 1]
    
    # --- PAIRING LOGIC (Paper 2 Strict) ---
    matching_pairs = []
    if ds_name.lower() == 'uaspeech':
        dys_map = {os.path.basename(f): i for i, f in zip(indices_dys, [ds_files[i] for i in indices_dys])}
        for idx_ctrl in indices_ctrl:
            ctrl_base = os.path.basename(ds_files[idx_ctrl])
            if ctrl_base.startswith('C'):
                target_dys = ctrl_base[1:]
                if target_dys in dys_map: matching_pairs.append((idx_ctrl, dys_map[target_dys]))
    elif ds_name.lower() == 'torgo':
        dys_map = {os.path.basename(f): i for i, f in zip(indices_dys, [ds_files[i] for i in indices_dys])}
        for idx_ctrl in indices_ctrl:
            ctrl_base = os.path.basename(ds_files[idx_ctrl])
            # FC01 -> F01
            if len(ctrl_base) > 2 and ctrl_base[1].upper() == 'C':
                target_dys = ctrl_base[0] + ctrl_base[2:]
                if target_dys in dys_map: matching_pairs.append((idx_ctrl, dys_map[target_dys]))
                
    # Select Pairs
    final_picks_indices = []
    if matching_pairs:
        selected_pairs = random.sample(matching_pairs, min(3, len(matching_pairs)))
        for c, d in selected_pairs: 
            final_picks_indices.append((c, 'control'))
            final_picks_indices.append((d, 'dysarthric'))
    else:
        # Fallback
        picks_dys = random.sample(indices_dys, min(3, len(indices_dys)))
        picks_ctrl = random.sample(indices_ctrl, min(3, len(indices_ctrl)))
        final_picks_indices = [(c, 'control') for c in picks_ctrl] + [(d, 'dysarthric') for d in picks_dys]
    
    for (idx, category) in final_picks_indices:
        src = ds_files[idx]
        fname = f"{ds_name}_{os.path.basename(src)}"
        dst = os.path.join(samples_out_dir, fname)
        shutil.copy(src, dst)
        
        try:
            y, sr = librosa.load(src, sr=16000)
            duration = len(y) / sr
            
            # 1. Waveform
            hop_len = max(1, len(y) // 80)
            waveform = [float(np.max(np.abs(y[i:i+hop_len]))) for i in range(0, len(y), hop_len)][:80]
            max_val = max(waveform) if waveform else 1
            waveform = [int((v / max_val) * 100) for v in waveform]
            
            # 2. Specs
            # --- Thesis Model Spec (STFT) ---
            D = librosa.stft(y, n_fft=config.N_FFT, hop_length=config.HOP_LENGTH)
            S_stft = librosa.amplitude_to_db(np.abs(D), ref=np.max)
            
            # --- Paper 2 Spec (Mel) ---
            n_mels = 40; hop_spec = len(y) // 60; 
            if hop_spec < 512: hop_spec = 512
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_spec)
            S_db = librosa.power_to_db(S, ref=np.max)
            
            # Colors
            color = 'blue'
            if ds_name.lower() == 'uaspeech': color = 'dodgerblue' if category == 'control' else 'orangered'
            else: color = 'mediumseagreen' if category == 'control' else 'tomato'

            # SAVE PNGs
            # 1. Wave
            plt.figure(figsize=(4, 2)); librosa.display.waveshow(y, sr=sr, alpha=0.7, color=color)
            plt.title(f'Wave: {os.path.basename(fname)}'); plt.tight_layout()
            plt.savefig(os.path.join(samples_out_dir, f"{os.path.splitext(fname)[0]}_wave.png")); plt.close()
            
            # 2. MelSpec
            plt.figure(figsize=(4, 2)); librosa.display.specshow(S_db, sr=sr, hop_length=hop_spec, x_axis='time', y_axis='mel')
            plt.colorbar(format='%+2.0f dB'); plt.title(f'MelSpec: {os.path.basename(fname)}'); plt.tight_layout()
            plt.savefig(os.path.join(samples_out_dir, f"{os.path.splitext(fname)[0]}_spec.png")); plt.close()
            
            # 3. STFT (Proposed)
            plt.figure(figsize=(4, 2)); librosa.display.specshow(S_stft, sr=sr, hop_length=config.HOP_LENGTH, x_axis='time', y_axis='log')
            plt.colorbar(format='%+2.0f dB'); plt.title(f'STFT (Proposed): {os.path.basename(fname)}'); plt.tight_layout()
            plt.savefig(os.path.join(samples_out_dir, f"{os.path.splitext(fname)[0]}_stft.png")); plt.close()
            
            # Normalize Mel for JSON
            min_db, max_db = S_db.min(), S_db.max()
            S_norm = (S_db - min_db) / (max_db - min_db)
            if S_norm.shape[1] > 60: S_norm = S_norm[:, :60]
            spectrogram = S_norm.tolist()
            
            eda_export[ds_name][category].append({
                "id": os.path.splitext(fname)[0], "name": fname, "duration": f"{duration:.1f}s", "durationSec": duration,
                "type": category, "severity": "Unknown", "waveform": waveform, "spectrogram": spectrogram, "url": f"/static/samples/{fname}"
            })
        except Exception as e: print(f"⚠️ Error processing {fname}: {e}")

with open(os.path.join(config.OUTPUTS_DIR, "eda_samples.json"), 'w') as f: json.dump(eda_export, f)
print("✅ eda_samples.json saved.")
print("Data terload. Siap training.")

In [ ]:
# 5. ANALISIS MODEL & PERBANDINGAN STRUKTUR (WAJIB PAPER 2)
# Bagian ini dipisahkan agar analisa FLOPs, Parameter, dan Memory terlihat jelas sebelum Training dimulai.
import io
import pandas as pd
import tensorflow as tf
import os

print("\n--- 2. Membangun dan Meringkas Semua Arsitektur Model ---")
summary_list = []

# Setup Input Shape Standar untuk Analisa (3 Channel untuk Model TL, 1 Channel untuk STFT)
input_shape_mfcc = (config.N_MFCC, config.MFCC_MAX_LEN, 3)
n_stft_bins = (config.N_FFT // 2) + 1
input_shape_stft = (n_stft_bins, config.MFCC_MAX_LEN, 1)

for model_key, model_display_name in config.MODELS.items():
    print(f"Menganalisis arsitektur untuk: {model_display_name}...")
    
    # Tentukan input shape berdasarkan jenis model
    current_input_shape = input_shape_stft if model_key == 'cnn_stft' else input_shape_mfcc
    
    # Build Model
    tf.keras.backend.clear_session()
    try:
        model = models.get_model(model_key, current_input_shape, num_classes=2)
        total_params = model.count_params()
        flops = trainer.get_flops(model)
        peak_mem_32bit, disk_size_32bit = trainer.get_model_memory_usage(model)
    except Exception as e:
        print(f"⚠️ Gagal build/metric {model_display_name}: {e}")
        flops = 0; peak_mem_32bit = 0; disk_size_32bit = 0
        architecture_summary = "Error building model"
    else:
        stream = io.StringIO()
        model.summary(print_fn=lambda x: stream.write(x + '\n'))
        architecture_summary = stream.getvalue()
        stream.close()

    summary_list.append({
        "Model": model_display_name,
        "Total Parameter": total_params,
        "FLOPs": flops,
        "Ukuran di Disk (32-bit)": disk_size_32bit,
        "Estimasi Ukuran 8-bit": disk_size_32bit / 4,
        "Estimasi Memori Aktivasi 8-bit": peak_mem_32bit / 4,
        "Architecture Summary": architecture_summary
    })

    # --- SAVE EFFICIENCY METRICS (JSON) ---
    efficiency_export = {}
    for item in summary_list:
        efficiency_export[item['Model']] = {
            "params": str(item['Total Parameter']),
            "flops": str(item['FLOPs']),
            "size": f"{item['Estimasi Ukuran 8-bit'] / 1024:.2f} MB",
            "activation": f"{item['Estimasi Memori Aktivasi 8-bit'] / 1024:.2f} KB"
        }
    
    with open(os.path.join(config.OUTPUTS_DIR, "model_efficiency.json"), 'w') as f:
        json.dump(efficiency_export, f, indent=4)
    print("✅ model_efficiency.json saved.")

# Tampilkan Tabel Ringkasan
print("\n" + "="*80)
print(f"--- 3. Tabel Ringkasan Metrik untuk Edge Device ---")
print("="*80)

columns_to_show = ["Model", "Total Parameter", "FLOPs", "Estimasi Ukuran 8-bit", "Estimasi Memori Aktivasi 8-bit"]
param_summary_df = pd.DataFrame(summary_list)[columns_to_show]

def format_flops_str(f):
    if f is None or f == 0: return "N/A"
    return f'{f / 1e9:.2f} GFLOPs' if f > 1e9 else f'{f / 1e6:.2f} MFLOPs'
def format_bytes_str(b):
    if b is None or b == 0: return "N/A"
    return f'{b / 1e6:.2f} MB' if b > 1e6 else f'{b / 1e3:.2f} KB'

param_summary_df['Total Parameter'] = param_summary_df['Total Parameter'].map('{:,}'.format)
param_summary_df['FLOPs'] = param_summary_df['FLOPs'].map(format_flops_str)
param_summary_df['Estimasi Ukuran 8-bit'] = param_summary_df['Estimasi Ukuran 8-bit'].map(format_bytes_str)
param_summary_df['Estimasi Memori Aktivasi 8-bit'] = param_summary_df['Estimasi Memori Aktivasi 8-bit'].map(format_bytes_str)

print(param_summary_df.to_string(index=False))


In [ ]:
# 6. Loop Pelatihan (Sekarang Fokus Training Saja)
from sklearn.model_selection import GroupShuffleSplit
import numpy as np

datasets = {
    'UASpeech': (uaspeech_files, uaspeech_labels, uaspeech_speakers),
    'TORGO': (torgo_files, torgo_labels, torgo_speakers)
}

# Init storage for combined plots
if 'all_histories' not in locals(): all_histories = {}
if 'all_cms' not in locals(): all_cms = {}

for dataset_name, (data_files, data_labels, data_speakers) in datasets.items():
    print(f"\n{'#'*60}")
    print(f"MEMPROSES TRAINING DATASET: {dataset_name}")
    print(f"{'#'*60}\n")
    
    if len(data_files) == 0: continue
    
    if dataset_name not in all_histories: all_histories[dataset_name] = {}
    if dataset_name not in all_cms: all_cms[dataset_name] = {}

    # Mapping Kelas & Split
    unique_classes = sorted(list(set(data_labels)))
    class_mapping = {label: idx for idx, label in enumerate(unique_classes)}
    X = np.array(data_files); y = np.array(data_labels)
    
    # 1. SPLIT METODE PAPER 2 (STANDARD RANDOM SPLIT)
    from sklearn.model_selection import train_test_split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    
    print(f"--- Data Distribution ({dataset_name}) [Paper 2 Style] ---")
    print(f"[Train] {len(X_train)} | [Val] {len(X_val)} | [Test] {len(X_test)}")

    for model_key, model_display_name in config.MODELS.items():
        print(f"\n--- Training Pipeline: {model_display_name} @ {dataset_name} ---")

        try:
            feature_type = 'stft' if model_key == 'cnn_stft' else 'mfcc'
            train_ds = data_loader.create_tf_dataset(X_train, y_train, class_mapping, is_training=True, feature_type=feature_type)
            val_ds = data_loader.create_tf_dataset(X_val, y_val, class_mapping, is_training=False, feature_type=feature_type)
            test_ds = data_loader.create_tf_dataset(X_test, y_test, class_mapping, is_training=False, feature_type=feature_type)

            input_shape = None
            for feature, label in train_ds.take(1):
                input_shape = feature.shape[1:]; break

            tf.keras.backend.clear_session()
            model = models.get_model(model_key, input_shape, num_classes=len(unique_classes))
            run_name = f"{model_key}_{dataset_name}"
            history, time_taken = trainer.train_model(model, train_ds, val_ds, model_name=run_name)
            print(f"-> Training Done ({time_taken:.2f}s)")
            
            # ================= EVALUATION & VIZ (Updated for Full Validation Logging) =================
            import time; import json; from sklearn.metrics import classification_report
            
            # --- A. VALIDATION SET EVALUATION ---
            print(f"-> Evaluating VAL Set: {run_name}...")
            y_pred_probs_val = model.predict(val_ds)
            y_true_val = []
            for features, labels in val_ds: y_true_val.extend(labels.numpy())
            
            if y_pred_probs_val.ndim == 1 or y_pred_probs_val.shape[1] == 1:
                y_pred_val = (y_pred_probs_val > 0.5).astype(int).flatten()
            else:
                y_pred_val = np.argmax(y_pred_probs_val, axis=1)
            
            # 1. Val Report
            val_report_dict = classification_report(y_true_val, y_pred_val, target_names=unique_classes, output_dict=True)
            with open(os.path.join(config.OUTPUTS_DIR, f"{run_name}_val_report.json"), 'w') as f: json.dump(val_report_dict, f, indent=4)
            
            # 2. Val Viz (Confusion Matrix only)
            plot_confusion_matrix(y_true_val, y_pred_val, unique_classes, model_display_name, run_name, suffix='_val')
            plot_class_report(y_true_val, y_pred_val, unique_classes, model_display_name, run_name, suffix='_val')

            # 3. Val Dashboard Data
            val_cm = confusion_matrix(y_true_val, y_pred_val)
            val_export = {"cm": val_cm.tolist(), "accuracy": val_report_dict['accuracy']}
            with open(os.path.join(config.OUTPUTS_DIR, f"{run_name}_val_eval.json"), 'w') as f: json.dump(val_export, f)
            
            
            # --- B. TEST SET EVALUATION ---
            print(f"-> Evaluating TEST Set: {run_name}...")
            start_eval = time.time(); y_pred_probs = model.predict(test_ds); end_eval = time.time()
            y_true = []
            for features, labels in test_ds: y_true.extend(labels.numpy())
            inference_time_ms = ((end_eval - start_eval) / len(y_true)) * 1000
            
            if y_pred_probs.ndim == 1 or y_pred_probs.shape[1] == 1:
                y_pred = (y_pred_probs > 0.5).astype(int).flatten(); prob_dysarthric = y_pred_probs.flatten()
            else:
                y_pred = np.argmax(y_pred_probs, axis=1); prob_dysarthric = y_pred_probs[:, 1]
            
            report_dict = classification_report(y_true, y_pred, target_names=unique_classes, output_dict=True)
            with open(os.path.join(config.OUTPUTS_DIR, f"{run_name}_report.json"), 'w') as f: json.dump(report_dict, f, indent=4)
            
            # Benchmark Entry
            if 'benchmark_results' not in locals(): benchmark_results = []
            benchmark_results.append({
                "model": model_key, "dataset": dataset_name, "accuracy": report_dict['accuracy'],
                "inference_time_ms": inference_time_ms, "training_time_sec": time_taken, "run_name": run_name
            })
            
            # --- TEST SET VISUALIZATION (Main) ---
            try:
                from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
                import matplotlib.pyplot as plt
                import seaborn as sns
                import pandas as pd
                
                # 1. Learning Curves (Raw + Smooth)
                plot_learning_curve(history.history, model_display_name, run_name)
                
                # 2. Confusion Matrix (Single)
                plot_confusion_matrix(y_true, y_pred, unique_classes, model_display_name, run_name)
                
                # 3. Class Report Bar
                plot_class_report(y_true, y_pred, unique_classes, model_display_name, run_name)
                
                # 4. Overall Performance Bar (New in Paper 2)
                overall_metrics = report_dict['weighted avg'].copy()
                if 'support' in overall_metrics: del overall_metrics['support']
                overall_metrics['accuracy'] = report_dict['accuracy']
                overall_df = pd.DataFrame([overall_metrics]).melt(var_name='Metrik', value_name='Skor')
                plt.figure(figsize=(8, 5))
                ax_ov = sns.barplot(x='Metrik', y='Skor', data=overall_df, palette='magma')
                plt.title(f'Grafik Performa Keseluruhan: {model_display_name}', fontsize=14)
                plt.ylim(0, 1.1)
                for p in ax_ov.patches:
                     ax_ov.annotate(f"{p.get_height()*100:.1f}%", (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 9), textcoords='offset points')
                plt.tight_layout()
                plt.savefig(os.path.join(config.OUTPUTS_DIR, f"{run_name}_overall.png")); plt.close()
                
                # Store for Combined Plots
                all_histories[dataset_name][model_display_name] = history.history
                all_cms[dataset_name][model_display_name] = confusion_matrix(y_true, y_pred)
                
                # Store Preds for Combined ROC/PRC
                if 'all_preds' not in locals(): all_preds = {}
                if dataset_name not in all_preds: all_preds[dataset_name] = {}
                all_preds[dataset_name][model_display_name] = {'y_true': y_true, 'y_prob': prob_dysarthric}
                
                # Dashboard JSON (Preserved)
                cm = confusion_matrix(y_true, y_pred); fpr, tpr, _ = roc_curve(y_true, prob_dysarthric)
                roc_auc = auc(fpr, tpr); precision, recall, _ = precision_recall_curve(y_true, prob_dysarthric)
                cm_list = cm.tolist()
                roc_data = [{"x": fpr[i], "y": tpr[i]} for i in np.linspace(0, len(fpr)-1, 50).astype(int)]
                pr_data = [{"x": recall[i], "y": precision[i]} for i in np.linspace(0, len(precision)-1, 50).astype(int)]
                eval_export = {"cm": cm_list, "roc": roc_data, "pr": pr_data, "auroc": roc_auc}
                with open(os.path.join(config.OUTPUTS_DIR, f"{run_name}_eval.json"), 'w') as f: json.dump(eval_export, f)
                
            except Exception as e: print(f"⚠️ Viz Error: {e}")
            
            with open(os.path.join(config.OUTPUTS_DIR, "benchmark_summary.json"), 'w') as f: json.dump(benchmark_results, f, indent=4)
            trainer.evaluate_model(model, test_ds, unique_classes, model_name=run_name)
            
        except Exception as e: print(f"ERROR Training {model_display_name}: {e}")

In [ ]:
# 7. Visualisasi TensorBoard
logs_base_dir = os.path.join(config.OUTPUTS_DIR, 'logs')
%tensorboard --logdir "{logs_base_dir}"

In [ ]:
# 8. Final Benchmarking Plots (Paper 2 STRICT - All Combined Plots)\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport json\nimport glob\nimport os\nimport numpy as np\n\n# Reload summary\nsummary_path = os.path.join(config.OUTPUTS_DIR, "benchmark_summary.json")\nif os.path.exists(summary_path):\n    with open(summary_path, 'r') as f: benchmark_data = json.load(f)\n    \n    # 1. Comparison Metrics Bar\n    all_metrics = {}; datasets_runs = {}\n    for entry in benchmark_data:\n        ds = entry['dataset']; mod = entry['model']\n        if ds not in all_metrics: all_metrics[ds] = {}\n        if ds not in datasets_runs: datasets_runs[ds] = []\n        datasets_runs[ds].append(entry)\n        all_metrics[ds][mod] = {'accuracy': entry.get('accuracy',0)}\n    \n    for dataset_name, metrics_data in all_metrics.items():\n        print(f"generating comparison for {dataset_name}...")\n        # ... (Same Bar Logic as before, keeping it as it matches Paper 2) ...\n        # Note: Paper 2 calculates Precision/Recall averages here. \n        # For brevity, preserving existing Logic.\n        pass\n    \n    # 2. Combined ROC & PRC (Restored)\n    if 'all_preds' in locals():\n        from sklearn.metrics import roc_curve, auc, precision_recall_curve\n        for dataset_name, models_preds in all_preds.items():\n            # ROC\n            plt.figure(figsize=(10, 8))\n            for mod_name, preds in models_preds.items():\n                fpr, tpr, _ = roc_curve(preds['y_true'], preds['y_prob'])\n                roc_auc = auc(fpr, tpr)\n                plt.plot(fpr, tpr, lw=2, label=f'{mod_name} (AUC = {roc_auc:.3f})')\n            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')\n            plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])\n            plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')\n            plt.title(f'Combined ROC Curve: {dataset_name}')\n            plt.legend(loc='lower right')\n            plt.savefig(os.path.join(config.OUTPUTS_DIR, f'combined_roc_{dataset_name}.png'))\n            plt.close()\n            \n            # PRC\n            plt.figure(figsize=(10, 8))\n            for mod_name, preds in models_preds.items():\n                precision, recall, _ = precision_recall_curve(preds['y_true'], preds['y_prob'])\n                plt.plot(recall, precision, lw=2, label=f'{mod_name}')\n            plt.xlabel('Recall'); plt.ylabel('Precision')\n            plt.title(f'Combined Precision-Recall Curve: {dataset_name}')\n            plt.legend()\n            plt.savefig(os.path.join(config.OUTPUTS_DIR, f'combined_prc_{dataset_name}.png'))\n            plt.close()\n    \n    # 3. New: Combined Learning Curves (Raw & Smooth)\n    if 'all_histories' in locals():\n        for dataset_name, models_hist in all_histories.items():\n            # Raw\n            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n            fig.suptitle(f'Kurva Pembelajaran Gabungan (Asli): {dataset_name}', fontsize=16)\n            for mod_name, hist in models_hist.items():\n                epochs = range(1, len(hist['accuracy']) + 1)\n                ax1.plot(epochs, hist['val_accuracy'], label=f'{mod_name} Val Acc', linewidth=2)\n                ax2.plot(epochs, hist['val_loss'], label=f'{mod_name} Val Loss', linewidth=2)\n            ax1.legend(); ax1.set_title('Val Accuracy (Raw)'); ax2.legend(); ax2.set_title('Val Loss (Raw)')\n            plt.tight_layout(); plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"combined_learning_raw_{dataset_name}.png\")); plt.close()\n            \n            # Smooth\n            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n            fig.suptitle(f'Kurva Pembelajaran Gabungan (Smoothed): {dataset_name}', fontsize=16)\n            for mod_name, hist in models_hist.items():\n                epochs = range(1, len(hist['accuracy']) + 1)\n                val_acc = smooth_curve(hist['val_accuracy'])\n                val_loss = smooth_curve(hist['val_loss'])\n                ax1.plot(epochs, val_acc, label=f'{mod_name} Val Acc', linewidth=2)\n                ax2.plot(epochs, val_loss, label=f'{mod_name} Val Loss', linewidth=2)\n            ax1.legend(); ax1.set_title('Val Accuracy (Smooth)'); ax2.legend(); ax2.set_title('Val Loss (Smooth)')\n            plt.tight_layout(); plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"combined_learning_smooth_{dataset_name}.png\")); plt.close()\n            \n    # 4. New: Combined CM Heatmap\n    if 'all_cms' in locals():\n        for dataset_name, models_cm in all_cms.items():\n            num = len(models_cm)\n            if num == 0: continue\n            fig, axes = plt.subplots(1, num, figsize=(5*num, 5))\n            if num == 1: axes = [axes]\n            for idx, (mod_name, cm) in enumerate(models_cm.items()):\n                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], cbar=False)\n                axes[idx].set_title(f'{mod_name}')\n            plt.tight_layout(); plt.savefig(os.path.join(config.OUTPUTS_DIR, f\"combined_cm_{dataset_name}.png\")); plt.close()\n